# Практическая работа 11.

In [145]:
import tqdm
import numpy as np
import pandas as pd
import time

from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN

import plotly.graph_objects as go
import plotly.express as px

## Задание 1.

Найти данные для кластеризации. Данные в группе не должны повторяться! Внимание, если признаки в данных имеют очень сильно разные масштабы, то необходимо данные предварительно нормализовать.

Возьмем датасет с [профессиями](https://www.kaggle.com/datasets/vetrirah/customer?select=Train.csv)

In [170]:
df = pd.read_csv("data/profession.csv").dropna().reset_index(drop=True)
df = pd.DataFrame(df, columns=['Age', 'Profession', 'Work_Experience', 'Family_Size'])

prof_list = list(set(df['Profession']))
for ind in range(len(df['Profession'])):
    df['Profession'][ind] = prof_list.index(df['Profession'][ind])

print(f"Список профессий: {prof_list}")
df

C:\Users\orono\AppData\Local\Temp\ipykernel_27796\2882232209.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Список профессий: ['Marketing', 'Engineer', 'Artist', 'Doctor', 'Homemaker', 'Entertainment', 'Lawyer', 'Executive', 'Healthcare']


,Age,Profession,Work_Experience,Family_Size
0,22,8,1.0,4.0
1,67,1,1.0,1.0
2,67,6,0.0,2.0
3,56,2,0.0,2.0
4,32,8,1.0,3.0
...,...,...,...,...
6660,41,2,0.0,5.0
6661,35,7,3.0,4.0
6662,33,8,1.0,1.0
6663,27,8,1.0,4.0


In [177]:
class Model:
    def __init__(self, dataframe: pd.DataFrame, label_column: str):
        self.model_kmeans = None
        self.model_agglomerating = None
        self.model_dbscan = None
        self.df = dataframe
        self.label_column = label_column
        self.clusters_num = 0
        self.score_value = 0
        self.silhouette = 0
        self.full_time=dict()

    def __call__(self, num_clusters: int, *args, **kwargs):
        self.k_means()
        self.agglomerating_model(clusters_num=num_clusters)
        self.dbscan_model(clusters_num=num_clusters)
        return self.full_time

    def k_means(self, show_results=False):
        df = self.df
        score_values = list()
        silhouette_list = list()

        for i in tqdm.tqdm(range(len(set(df[self.label_column])) - 1, 20), desc="Find the optimal num of clusters"):
            self.model_kmeans = KMeans(n_clusters=i,init='k-means++').fit(df.drop(columns=self.label_column))
            if silhouette_score(df, self.model_kmeans.labels_) > self.silhouette:
                self.silhouette = silhouette_score(df, self.model_kmeans.labels_)
                self.clusters_num = i
            score_values.append(self.model_kmeans.inertia_)
            silhouette_list.append(silhouette_score(df, self.model_kmeans.labels_))

        start_time = time.time()
        self.model_kmeans = KMeans(n_clusters=self.clusters_num,init='k-means++').fit(df.drop(columns=self.label_column))
        end_time = time.time()
        self.full_time['kmeans'] = end_time-start_time

        if show_results:
            labels = self.model_kmeans.labels_
            df['Cluster_kmeans'] = labels
            df = df.drop(columns=self.label_column)
            data_scores = {'n_clusters': np.arange(len(set(self.df[self.label_column])) - 1, 20),
                           'cost_function': score_values,
                           'silhouette': silhouette_list}

            data_scores = pd.DataFrame(data_scores)
            fig = px.line(data_scores, x="n_clusters", y="cost_function", title="Функция стоимости", markers=True)
            fig.show()
            fig = px.line(data_scores, x="n_clusters", y="silhouette", title="Коэффициенты силуэта", markers=True)
            fig.show()
            print(f"Лучшее число кластеров: {self.clusters_num}")
            self._plot_results(column_name='Cluster_kmeans', model=self.model_kmeans)
        return self.model_kmeans, self.clusters_num

    def agglomerating_model(self, show_results=False, clusters_num=10):
        start_time = time.time()
        self.model_agglomerating = AgglomerativeClustering(clusters_num, compute_distances=True).fit(self.df.drop(columns=self.label_column))
        end_time = time.time()
        self.full_time['agglomerating'] = end_time-start_time
        if show_results:
            self._plot_results(column_name='Cluster_agglomerating', model=self.model_agglomerating)
        return self.model_agglomerating

    def dbscan_model(self, show_results=False, clusters_num=10, eps=0.5):
        start_time = time.time()
        self.model_dbscan = DBSCAN(eps= eps, min_samples=clusters_num).fit(self.df.drop(columns=self.label_column))
        end_time = time.time()
        self.full_time['dbscan'] = end_time-start_time
        if show_results:
            self._plot_results(column_name='Cluster_dbscan', model=self.model_dbscan)
        return self.model_dbscan

    def _plot_results(self, column_name: str, model):
        self.df[column_name] = model.labels_
        fig = go.Figure(data=[go.Scatter3d(x=self.df['Age'], y=self.df['Work_Experience'], z=self.df['Family_Size'],
                                       mode='markers', marker=dict(color=self.df[column_name], size=4))])
        fig.update_layout(scene = dict(
                    xaxis_title=self.df.keys()[0],
                    yaxis_title=self.df.keys()[2],
                    zaxis_title=self.df.keys()[3]),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))
        fig.show()

## Задание 2.

Провести кластеризацию данных с помощью алгоритма k-means. Использовать «правило локтя» и коэффициент силуэта для поиска оптимального количества кластеров.

In [178]:
model = Model(dataframe=df, label_column='Profession')
num_clusters = model.k_means(show_results=True)[1]

Find the optimal num of clusters: 100%|██████████| 12/12 [00:20<00:00,  1.69s/it]


Лучшее число кластеров: 9


## Задание 3.

Провести кластеризацию данных с помощью алгоритма иерархической кластеризации.

In [179]:
model.agglomerating_model(show_results=True, clusters_num=num_clusters)

AgglomerativeClustering(compute_distances=True, n_clusters=9)

## Задание 4.

Провести кластеризацию данных с помощью алгоритма DBSCAN..
eps - Максимальное расстояние между двумя образцами, при котором один считается соседним с другим. Это не максимальная граница расстояний между точками внутри кластера. Это самый важный параметр DBSCAN, который следует правильно выбрать для вашего набора данных и функции расстояния.
clusters_num - Количество выборок (или общий вес) в окрестности точки, которую следует рассматривать как основную точку. В том числе и сама точка.

In [180]:
model.dbscan_model(show_results=True, clusters_num=num_clusters, eps=4)

DBSCAN(eps=4, min_samples=9)

## Задание 5.

Сравнить скорость работы алгоритмов. Результаты изобразить в виде таблицы.

In [181]:
times = Model(dataframe=df, label_column='Profession')(num_clusters=num_clusters)

Find the optimal num of clusters: 100%|██████████| 12/12 [00:19<00:00,  1.66s/it]


In [182]:
pd.DataFrame(times, index=['time (seconds)'])

,kmeans,agglomerating,dbscan
time (seconds),0.061061,1.202649,0.084788
